In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Packages

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from itertools import groupby
%matplotlib inline
import tensorflow as tf
import tensorflow_hub as hub
from scipy.stats import rankdata


# Import Data

In [3]:
df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/raw_data_preprocessed.csv')
df.head(5)

,question_id,question_title_raw,question_body_raw,question_body,question_body_text,question_body_code,question_title_body_raw,question_title_body,question_title_body_text,question_title_body_code,answer_id,answer_body_raw,answer_body,answer_text,answer_code,answer_count,view_count,answer_score,is_accepted_answer,question_title_body_order_textcode,question_title_body_order_codetext,answer_body_order_textcode,answer_body_order_codetext
0,55661532,"When I run my socket program, there get: `impo...",<p>I write a socket server programmer:</p>\n\n...,I write a socket server programmer:\n#-*- codi...,I write a socket server programmer:\n\nwhen I ...,#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,"When I run my socket program, there get: `impo...","When I run my socket program, there get: `impo...","When I run my socket program, there get: `impo...",#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,55661630,<p>The answer to your new question is that you...,The answer to your new question is that you ca...,The answer to your new question is that you ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE...",2,1816,0,0,"When I run my socket program, there get: `impo...",#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,The answer to your new question is that you ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE..."
1,55661626,"with socket.socket(socket.AF_INET, socket.SOCK...",<p>I have a socket server program:</p>\n\n<pre...,I have a socket server program:\n#!/usr/bin/en...,I have a socket server program:\n\nwhen I run ...,#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,"with socket.socket(socket.AF_INET, socket.SOCK...","with socket.socket(socket.AF_INET, socket.SOCK...","with socket.socket(socket.AF_INET, socket.SOCK...",#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,55661660,<p>You can't use <code>socket.socket(socket.AF...,"You can't use socket.socket(socket.AF_INET, so...",You can't use with . So that a statement ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE...",2,2521,0,1,"with socket.socket(socket.AF_INET, socket.SOCK...",#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,You can't use with . So that a statement ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE..."
2,55661929,How to display 5 numbers per line from a list?,<p>How to display 5 numbers per line from a li...,How to display 5 numbers per line from a list?...,How to display 5 numbers per line from a list?...,"lx = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,...",How to display 5 numbers per line from a list?...,How to display 5 numbers per line from a list?...,How to display 5 numbers per line from a list?...,"lx = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,...",55662134,<p>There can be different approaches to the pr...,There can be different approaches to the probl...,There can be different approaches to the probl...,x=[] #your given list\nfor t in range(len(x))...,4,125,0,0,How to display 5 numbers per line from a list?...,"lx = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,...",There can be different approaches to the probl...,x=[] #your given list\nfor t in range(len(x))...
3,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,I want to print > in the script 0 to test the ...,I want to print in the script 0 to test the c...,>script[0][4][2]name 'script' is not defined\n...,create list with spectial type - Spyder error ...,create list with spectial type - Spyder error ...,create list with spectial type - Spyder error ...,>script[0][4][2]name 'script' is not defined\n...,55662821,<p>Define list as <code>[]</code> and use <cod...,Define list as [] and use append to add elemen...,Define list as and use to add elements to li...,"[]appendscript = []\n\nscript.append(['A','A',...",2,24,0,1,create list with spectial type - Spyder error ...,>script[0][4][2]name 'script' is not defined\n...,Define list as and use to add elements to li...,"[]appendscript = []\n\nscript.append(['A','A',..."
4,55662778,create

In [4]:
#Data Cleaning
df = df.fillna("")
df = df.replace("\n", " ", regex=True)

In [15]:
question_rows = df.loc[df['question_id'] == 55661929]
question = [question_rows['answer_body'].iloc[2]]

print(question)

["You can make the function a generator by making it yield the sliced lists instead: def display(lx):     for i in range(0, len(lx), 5):         yield lx[i:i + 5]  print(*display(lx), sep='\\n')  This outputs: [1, 2, 4, 5, 6] [7, 8, 9, 10, 11] [12, 13, 14, 15, 16] [17, 18, 19, 20, 21] [22, 23, 24, 25] "]


# Import Global Dictionaries

In [ ]:
# Load q_a_dictionaries from file
train_from_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/q_a_dict_train.pickle', 'rb')
train_qa_dict = pickle.load(train_from_file)

# Load q_a_dictionaries from file
dev_from_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/q_a_dict_dev.pickle', 'rb')
dev_qa_dict = pickle.load(dev_from_file)

# Load q_a_dictionaries from file
test_from_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/q_a_dict_test.pickle', 'rb')
test_qa_dict = pickle.load(test_from_file)

In [ ]:
train_qa_dict[53804101]

[(53804190, 0),
 (53804192, 0),
 (53804260, 1),
 (50244366, 0),
 (56139960, 0),
 (62474012, 0),
 (52345915, 0),
 (56674991, 0),
 (39559140, 0),
 (37970415, 0)]

# Create Embeddings Dictionary for Question Title Body and Answer Body

In [ ]:
#Load Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

1) embedding dictionary
{'id' => string 'embedding' (bert and use)}
id - unique question ids + answer ids = 90,000 something ids total
need to create multiple dictionaries for each type of embedding that we want

In [ ]:
tester = "Elephant"
message = [tester]
message_embedding = embed(message)
print(message_embedding)

tf.Tensor(
[[ 8.34446214e-03  4.80855932e-04  6.59524798e-02 -1.97126511e-02
   3.57008167e-02  3.65892835e-02  3.70612629e-02 -1.12046087e-02
  -3.17393546e-03  4.21421938e-02  7.64983818e-02  7.04209059e-02
  -1.62738729e-02 -2.85084196e-03 -3.38686332e-02 -5.61367981e-02
  -3.20003144e-02  3.03572211e-02 -7.99227059e-02 -5.48588075e-02
   2.84989271e-02  3.66121419e-02  5.85439242e-02  1.70133542e-02
  -5.32833226e-02  3.42066549e-02 -3.17066051e-02 -6.32714406e-02
   1.71268079e-02 -3.16850618e-02 -2.44463924e-02 -7.80240372e-02
   4.53712940e-02 -2.56830305e-02 -7.20223784e-02  3.57165933e-02
  -4.69715260e-02  7.40225986e-02  7.19125196e-03  7.71081727e-03
   3.19260582e-02  1.70774627e-02 -2.34687831e-02 -3.52300331e-02
  -8.00300092e-02  3.54695246e-02 -3.86509225e-02  8.14682804e-03
  -1.60237011e-02  4.43639159e-02  7.98752382e-02 -1.94660109e-02
   7.43112937e-02  5.95273226e-02 -3.31025128e-03 -5.19299228e-03
   4.56661247e-02 -6.02190103e-03  5.58695234e-02 -5.20962812e-02

In [ ]:
#@title Compute a representation for each message, showing various lengths supported.
word = "Elephant"
# sentence = "I am a sentence for which I would like to get its embedding."
# paragraph = (
#     "Universal Sentence Encoder embeddings also support short paragraphs. "
#     "There is no hard limit on how long the paragraph is. Roughly, the longer "
#     "the more 'diluted' the embedding will be.")
messages = [word]

# # Reduce logging output.
# logging.set_verbosity(logging.ERROR)

message_embeddings = embed(messages)

print(len(message_embeddings))

1


In [ ]:
# Compute a representation for each message, showing various lengths supported.
messages = ["That band rocks!", "That song is really cool."]

embed(messages)

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[ 0.01565756,  0.03393447,  0.09912308, ..., -0.02306146,
        -0.06473583, -0.02028312],
       [ 0.05407216,  0.03738671,  0.10798929, ..., -0.02737424,
        -0.02487121, -0.0030675 ]], dtype=float32)>

In [ ]:
#Get distinct questions and answer ids
distinct_question_ids = df['question_id'].unique()
distinct_answer_ids = df['answer_id'].unique()

In [ ]:
#Function to create USE embeddings for questions
def use_embedding_question_dictionary(df):
  use_question_dictionary = {}
  #Loop through unique question ids
  for id in distinct_question_ids[0:1]:
    #Take the id and get the question rows
    question_rows = df.loc[df['question_id'] == id]

    #Take just one row and get the question
    question = [question_rows['question_title_body'].iloc[0]]

    #Embed
    question_embedding = embed(question)

    #Add to dictionary
    use_question_dictionary[id] = question_embedding

  return use_question_dictionary

In [ ]:
#Function to create USE embeddings for answers
def use_embedding_answer_dictionary(df):
  use_answer_dictionary = {}
  #Loop through unique question ids
  for id in distinct_answer_ids:
    #Take the id and get the question rows
    answer_rows = df.loc[df['answer_id'] == id]

    #Take just one row and get the question
    answer = [answer_rows['answer_body'].iloc[0]]

    #Embed
    answer_embedding = embed(answer)

    #Add to dictionary
    use_answer_dictionary[id] = answer_embedding

  return use_answer_dictionary

In [ ]:
use_question_dict = use_embedding_question_dictionary(df)
use_answer_dict = use_embedding_answer_dictionary(df)

<class 'list'>
(1, 512)


In [ ]:
#Combine the 2 dictionaries
use_embeddings_dict = use_question_dict.copy()
use_embeddings_dict.update(use_answer_dict)

# Save embeddings dictionary to file
use_embeddings_dict_to_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/embeddings/use_embeddings.pickle', 'wb')
pickle.dump(use_embeddings_dict, use_embeddings_dict_to_file)

# Load Embeddings Dictionary

In [ ]:
# Load embeddings dictionary from file
use_embeddings_from_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/embeddings/use_embeddings.pickle', 'rb')
loaded_embeddings = pickle.load(use_embeddings_from_file)

#Create Dot Product Dictionaries

In [ ]:
# Import similarity_score_dicts.py file
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/master/helper_functions')
import similarity_score_dicts

In [ ]:
dot_product_dict = similarity_score_dicts.create_similarity_score_dict(test_qa_dict, loaded_embeddings, 'dot_product')

In [ ]:
dot_product_ranking_dict = similarity_score_dicts.compute_rankings(dot_product_dict)


#Evaluate Results

In [ ]:
# Import evaluation.py file
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/master/helper_functions')
import evaluation

In [ ]:
evaluation.calculate_mrr(test_qa_dict, dot_product_ranking_dict)

MRR: 0.7566854002456338


0.7566854002456338

In [ ]:
evaluation.calculate_metrics(test_qa_dict, dot_product_ranking_dict)

TP: 1869
FP: 1191
TN: 26373
FN: 1197
Precision: 0.6107843137254902
Recall: 0.6095890410958904
Accuracy: 0.9220372184133203


In [ ]:
dot_product_ranking_dict

{55661929: [(55662134, 2),
  (55662053, 4),
  (55661981, 1),
  (64717925, 8),
  (52250747, 7),
  (47805233, 6),
  (60398311, 3),
  (47860893, 5),
  (50565510, 9),
  (54576739, 10)],
 55664544: [(55664726, 4),
  (55664585, 1),
  (55664663, 2),
  (51611111, 8),
  (55118801, 6),
  (61665527, 10),
  (58582708, 3),
  (59407189, 9),
  (61027439, 7),
  (42335581, 5)],
 55644043: [(55665839, 1),
  (55644071, 2),
  (55644063, 3),
  (43736281, 10),
  (56789874, 9),
  (61598381, 5),
  (38446230, 8),
  (57796365, 4),
  (60590369, 7),
  (37491434, 6)],
 55668648: [(55669044, 2),
  (55668752, 1),
  (40313673, 6),
  (61019118, 8),
  (42076860, 5),
  (46987965, 3),
  (42858938, 7),
  (55284252, 9),
  (62841773, 4),
  (42798604, 10)],
 55707618: [(55708016, 1),
  (62266543, 9),
  (41499488, 4),
  (51143548, 8),
  (35772065, 2),
  (50058945, 7),
  (38295966, 3),
  (46439272, 6),
  (56278456, 5),
  (63433599, 10)],
 55741792: [(55750129, 1),
  (55742051, 2),
  (62604102, 9),
  (49949152, 7),
  (61946349,